# Curve Fitting
Curve fitting is an optimization process of constructing a curve, or mathematical function, that has the best fit to a series of data points that may be subject to constraints. The process of curve fitting can involve interpolation, the process of estimating new data points in the range of a discrete set of known data points. Curve fitting can assist with extrapolation, the process of estimating new data points outside of the range of known data points.

Python implements curve fitting through a function located in the `scipy.optimize` package. By using the function `curve_fit`, you can define a model fit function and method of fit, and obtain the best fit parameters and a covariance matrix for the parameters.

## Methods of Fit
`curve_fit` uses one of 3 methods to fit data. The fitting method is specified in the `curve-fit` function using the argument, `method='lm'`. 

### `lm`
The default method is the Levenberg-Marquardt algorithm, also called the damped least-squares. The LM method interpolates between the Gauss-Newton algorithm and the method of gradient descent. In general terms, given $m$ pairs of data $(x_i,y_i)$, it finds the parameter(s) $\beta$ of the model function $f(x,\beta)$ that minimizes the sum of the squares of the deviations, that is 
$$\sum_{i=1}^{m}[y_i-f(x_i,\beta)]^2$$
The algorithm does this through an iterative process. Note that this algorithm will no work if the number of points $m$ is less than the number of parameters in the function.

### `trf`
The second method is known as the *trust region method*. This method fits the parameters using a least-squares-like measurement in defined regions of the data. These regions expand and contract based on the goodness of fit in the current region. These sets of algorithms are also referred to as *restricted-step methods*

### `dogbox`
The last method is the *Powell's dog leg method*. This method is an iterative optimisation algorithm silmilar to the Levenberg-Marquardt algorithm, but it uses a explicit trust region in each iteration. In each step, it searches for a minimum in the objective functions along the steepest descent direction. Its name comes from the resemblance between the construction of the dog leg step and the shape of a [dogleg hole](https://en.wikipedia.org/wiki/Golf_course#Fairway_and_rough) in golf


## How do you go about fitting the statistical model to the data?

Let's consider some alpha particle decay data, it is contained in the `alphadecay.txt` file. The file contains 3 columns: time of measurement (in days), rate of decay (in detections per second), and the uncertainty in the rate of decay. The rate of decay is described by the function
$$R(t) = R_0 e^{-t/\tau}$$  
where $\tau$ is the mean lifetime of the alpha particle. In an ideal world, we could isolate the laboratory completely, however, some of the measured alpha decays in the data set are from background radioactivity. Instead, we can describe this data by adding a parameter,
$$R(t) = R_0 e^{-t/\tau} + R_{bkgd}$$



In [ ]:
import numpy as np
import pandas as pd
import scipy.optimize as sy

decayfile = pd.read_table('data/alphadecay.txt', sep= '\s+', header=None)
x_time = decayfile.loc[:,0]
y_rate = decayfile.loc[:,1]
#define fit function
def decayfit(x, a, b, c):
    return a*np.exp(-x/b)+c
#fit 
param, covar = sy.curve_fit(decayfit, x_time, y_rate, p0=[2.1892,1,0.2])


In [ ]:
#mean lifetime
param[1]

In [ ]:
#covariance matrix
covar

In [ ]:
import matplotlib.pyplot as mp
#plot data and fit function
mp.scatter(x_time, y_rate)
mp.plot(x_time, decayfit(x_time,param[0],param[1],param[2]),'r')
#plot constant background
mp.plot(x_time,param[2]*np.ones(len(x_time)),'k--')

## Covariant Matrix
The second parameter returned by `curve_fit` is the [covariance matrix](https://en.wikipedia.org/wiki/Estimation_of_covariance_matrices) for the fit parameters. The covariance matrix is a measure of the uncertainty in the parameter estimates. It is a square matrix with dimensions equal to the number of parameters in the fit function. The diagonal elements of the matrix are the variances of the individual parameters, or the uncertainty in those estimates. The off-diagonal elements are the covariances between the parameters, which represent the degree of correlation between the estimates of those parameters. The elements themselves hold very little information, but statistics derived from them are useful.

In [ ]:
covar

The correlation between two parameters can be calculated. This tells you how independent they are.

In [ ]:
Rsquared = covar[0,2]/np.sqrt(covar[0,0]*covar[2,2])
print(Rsquared)

# Particle Physics Example
Let's look through the file `dimuon.csv`. This file contains data about particle collisions resulting in two muons. The columns contain the energy, momentum, and charge of each muon pair. The last column `M` contains the invariant mass calculation of the parent particle in the interaction: $X\rightarrow \mu^+\mu^-$. By analyzing the invariant mass values, you can find the identity of particle $X$.  

In [ ]:
import pandas as pd
data = pd.read_csv('data/dimuon.csv')

In [ ]:
data

In [ ]:
#look at the data
data.hist('M',bins=1000)

In [ ]:
#make a cut on the data
data[(data.M<3.6) & (data.M>2.5)].hist('M',bins=1000)

In [ ]:
focus = data[(data.M<3.6) & (data.M>2.5)]

In [ ]:
#define the fit to Breit-Wigner function
def bw(x,k,M,g):
    return k/((x**2-M**2)**2 + g**2*M**2)

In [ ]:
#get parameters of the histogram to put in curve fit
counts, bins = np.histogram(focus.M,bins=1000)
newbins = np.zeros(len(counts))
for k in range(0,len(bins)-1):
    newbins[k] = (bins[k]+bins[k+1])/2

In [ ]:
muparams,mucovar = sy.curve_fit(bw,newbins,counts,p0 = [300,3.9,0.25],method='lm')

In [ ]:
muparams

In [ ]:
mucovar

In [ ]:
#see how the fit appears in graphs
mp.plot(newbins,bw(newbins,muparams[0],muparams[1],muparams[2]))
mp.hist(focus.M,bins=1000)

In [ ]:
data[data.pt2<20].hist('pt2',bins=1000)

# Is this curve fitting or linear regression?
The distinction between curve fitting and linear regression is fuzzy. In my opinion, curve fitting becomes linear regression when you have a hypothesis about how the data is generated (what physical phenomena is producing the data) and how the parameters relate to one another, and when you dig deeper into the statistical analysis of the parameters and the data.